In [3]:
import numpy as np
import pandas as pd

csv_file_path = '../data/LAO/WHO_LAO_adm_lvl1_dengue_1998-2007.csv'

In [4]:
df = pd.read_csv(csv_file_path, header=None, sep=",")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,NaN,NaN,NaN,NaN,1998 month: 1,1998 month: 2,1998 month: 3,1998 month: 4,1998 month: 5,1998 month: 6,...,2006 month: 5,2006 month: 6,2006 month: 7,2006 month: 8,2006 month: 9,2006 month: 10,2006 month: 11,2006 month: 12,2007 month: 4,2007 month: 10
1,LAO,Lao People's Democratic Republic,LA.AT,Attapu,0,0,0,0,0,0,...,0,0,1,0,41,199,0,0,n/a,n/a
2,LAO,Lao People's Democratic Republic,LA.BK,Bokeo,0,0,0,0,0,0,...,0,2,1,4,2,5,0,0,n/a,n/a
3,LAO,Lao People's Democratic Republic,LA.BL,Bolikhamxai,0,2,0,0,80,7,...,9,18,24,75,51,21,26,4,n/a,n/a
4,LAO,Lao People's Democratic Republic,LA.CH,Champasak,1,1,12,10,0,17,...,53,134,221,182,55,127,159,161,n/a,n/a


In [5]:
def convert_date(raw_date):
    import re
    result = None
    date_regex = r'(\d{4})\s+month:\s+(\d{1,2})'
    matches = re.search(date_regex, raw_date)
    
    if matches:
        year = matches.group(1)
        month = matches.group(2)
        if len(month) == 1:
            month = '0' + month
        result = '{0}{1}01'.format(year, month)
    
    return result

In [6]:
arr = df.as_matrix().T

dates_raw = arr[4:, 0]
dates = [convert_date(x) for x in dates_raw]

places_raw = arr[:4, :]

new_arr = np.array([])

for i in range(1, arr.shape[1]):
    place_data = arr[:4, i]
       
    dengue_data = arr[4:, i]
    for j in range(dengue_data.shape[0]):
        date = dates[j]
        dengue_cases = dengue_data[j]
        if dengue_cases == 'n/a':
            dengue_cases = -1
        
        new_row = np.array(place_data)
        new_row = np.append(new_row, date)
        new_row = np.append(new_row, dengue_cases)
        
        if new_arr.shape[0] == 0:
            new_arr = new_row
        else:
            new_arr = np.vstack((new_arr, new_row))

        
column_names = ['country_iso2_id', 'country_name', 'admin_lvl1_id', 'admin_lvl1_name', 'period_date', 'dengue_total_cases']
new_df = pd.DataFrame(new_arr, columns=column_names)
new_df = new_df.apply(lambda x: pd.to_numeric(x, errors='ignore'))

new_df.head()

,country_iso2_id,country_name,admin_lvl1_id,admin_lvl1_name,period_date,dengue_total_cases
0,LAO,Lao People's Democratic Republic,LA.AT,Attapu,19980101,0
1,LAO,Lao People's Democratic Republic,LA.AT,Attapu,19980201,0
2,LAO,Lao People's Democratic Republic,LA.AT,Attapu,19980301,0
3,LAO,Lao People's Democratic Republic,LA.AT,Attapu,19980401,0
4,LAO,Lao People's Democratic Republic,LA.AT,Attapu,19980501,0


In [7]:
new_df.to_csv('lao_denguenet_data_refactored.csv', index=False, header=True)